In [12]:
import tweepy
import json
import re
import arrow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
from dateutil.parser import parse

# set up

In [13]:
# get access to twitter's api using own account

CONSUMER_KEY    = 'P75Ch94wYjCV4mk1x4yg2lYM4'
CONSUMER_SECRET = 'XZx0frpcXcqUbTT82OuxlEHV9453D4xnDcozZVnSbpPPs9KEBM'

ACCESS_TOKEN  = '933380984535334913-YgfWZKzumPC3RJzgQn2ts04QPhnF2fH'
ACCESS_TOKEN_SECRET = 'oeji3i3YAU5JkZ5OAorNMZYrzKJXPy0o0m9CXGOGuThwV'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

# collect data from 2019-05-01 00:00:00 EST to 2020-04-30 23:59:59 EST
start_date = parse('2019-05-01 04:00:00') # UTC
end_date = parse('2020-05-01 04:00:00') # UTC, exclusive

In [53]:
def save_data_json(data, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def save_data_csv(df, file_name):
    df.to_csv(file_name, sep=',', encoding='utf-8', index=False)
    
def load_data_json(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# get tweets from Twitter API

In [15]:
# get the latest 3200 tweets of Trump from twitter's api

def get_data_from_user_timeline():
    max_id = None
    count = 0
    trump_json = {}
    while(True):
        tweets = api.user_timeline('realDonaldTrump', count=400, max_id = max_id, tweet_mode='extended')
        if len(tweets) == 0: 
            break
        max_id = tweets[-1].id
        count =+ len(tweets)
        for tweet in tweets: 
            if tweet.created_at < end_date and tweet.created_at >= start_date:
                trump_json[tweet.id_str] = tweet._json
        if (count >= 3200): 
            break
    return trump_json

In [4]:
trump_json = get_data_from_user_timeline()
save_data_json(trump_json, 'trump_new.json')

# load archived tweets and get full tweets from Twitter API

In [24]:
# archive source: http://www.trumptwitterarchive.com/archive

def get_data_from_get_status(status_id):
    tweet = api.get_status(status_id, tweet_mode='extended')
    return tweet

def get_data_from_archive(trump_json):
    archive_error = load_data_json('trump_archive_error.json')
    archive_error_ids = list(map(lambda x: x['id_str'], archive_error))
    for a in load_data_json('trump_archive.json'):
        if a['id_str'] not in trump_json and a['id_str'] not in archive_error_ids:
            try:
                tweet = get_data_from_get_status(a['id_str'])
                if tweet.created_at < end_date and tweet.created_at >= start_date:
                    trump_json[a['id_str']] = tweet._json
            except tweepy.TweepError as e:
                print(a['id_str'], e)
                if e.args[0][0]['code'] == 88:
                    break
                else:
                    archive_error.append({'id_str': a['id_str'], 'code': e.args[0][0]['code'], 'message': e.args[0][0]['message']})
                    continue
    save_data_json(archive_error, 'trump_archive_error.json')

In [26]:
# trump_json = load_data_json('trump_new.json')
get_data_from_archive(trump_json)
save_data_json(trump_json, 'trump_new.json')

# get in-reply-to tweets from Twitter API

In [36]:
def get_data_in_reply_to(trump_json):
    trump_in_reply_to_json = {}
    for i, t in trump_json.items():
        if t['in_reply_to_status_id']:
            try:
                tweet = get_data_from_get_status(t['in_reply_to_status_id'])
                trump_in_reply_to_json[t['in_reply_to_status_id']] = tweet._json
            except tweepy.TweepError as e:
                print(t['in_reply_to_status_id'], e)
                if e.args[0][0]['code'] == 88:
                    break
                else:
                    continue
    return trump_in_reply_to_json

In [32]:
# trump_json = load_data_json('trump_new.json')
trump_in_reply_to_json = get_data_in_reply_to(trump_json)
save_data_json(trump_in_reply_to_json, 'trump_in_reply_to.json')

1209462718899638273 [{'code': 144, 'message': 'No status found with that ID.'}]
1206440080220606464 [{'code': 144, 'message': 'No status found with that ID.'}]
1138782960444956673 [{'code': 144, 'message': 'No status found with that ID.'}]


# convert tweets from json to csv

In [48]:
# convert utc to est
def convertUTCtoEST(date):
    utc = arrow.get(parse(date))
    return utc.to('US/Eastern').format('YYYY-MM-DD HH:mm:ss ZZ')

# convert to year
def convertUTCtoYear(date):
    utc = arrow.get(parse(date))
    return utc.to('US/Eastern').format('YYYY')

# convert to month
def convertUTCtoMonth(date):
    utc = arrow.get(parse(date))
    return utc.to('US/Eastern').format('MM')

# convert to day
def convertUTCtoDay(date):
    utc = arrow.get(parse(date))
    return utc.to('US/Eastern').format('DD')

# convert to hour
def convertUTCtoHour(date):
    utc = arrow.get(parse(date))
    return int(utc.to('US/Eastern').format('HH'))

# convert to day of week
def convertUTCtoDayOfWeek(date):
    utc = arrow.get(parse(date))
    return utc.to('US/Eastern').format('d')

# convert to week of year
def convertUTCtoWeek(date):
    return int(parse(date).strftime('%U'))

def convert_data_to_df(trump_json, trump_in_reply_to_json):
    trump_list = []
    for _, t in trump_json.items():
        t_new = {}
        t_new['id'] = t['id']
        t_new['created_at'] = t['created_at']
        t_new['source'] = re.sub(r'<.*?>', '', t['source'])
        t_new['year'] = convertUTCtoYear(t['created_at'])
        t_new['month'] = convertUTCtoMonth(t['created_at'])
        t_new['day'] = convertUTCtoDay(t['created_at'])
        t_new['hour'] = convertUTCtoHour(t['created_at'])
        t_new['day_of_week'] = convertUTCtoDayOfWeek(t['created_at'])
        t_new['week'] = convertUTCtoWeek(t['created_at'])
        if t['in_reply_to_status_id']:
            t_new['is_reply'] = True
            t_new['original_text'] = trump_in_reply_to_json[t['in_reply_to_status_id']]['full_text'] if t['in_reply_to_status_id'] in trump_in_reply_to_json else None
            t_new['original_author'] = t['in_reply_to_screen_name']
        else:
            t_new['is_reply'] = False
        if 'retweeted_status' in t:
            # retweet without comment
            t_new['is_retweet'] = True
            t_new['original_text'] = t['retweeted_status']['full_text']
            t_new['original_author'] = t['retweeted_status']['user']['screen_name']
        elif 'quoted_status' in t:
            # retweet with comment
            t_new['is_retweet'] = True
            t_new['text'] = t['full_text']
            t_new['favorite_count'] = t['favorite_count']
            t_new['retweet_count'] = t['retweet_count']
            t_new['original_text'] = t['quoted_status']['full_text']
            t_new['original_author'] = t['quoted_status']['user']['screen_name']
        else:
            t_new['is_retweet'] = False
            t_new['text'] = t['full_text']
            t_new['favorite_count'] = t['favorite_count']
            t_new['retweet_count'] = t['retweet_count']
        trump_list.append(t_new)
    return pd.DataFrame(trump_list)

In [49]:
# trump_json = load_data_json('trump_new.json')
trump_df = convert_data_to_df(trump_json, trump_in_reply_to_json)

In [52]:
trump_df.head()

,id,created_at,source,year,month,day,hour,day_of_week,week,is_reply,is_retweet,original_text,original_author,text,favorite_count,retweet_count
0,1256037319963967488,Fri May 01 01:46:47 +0000 2020,Twitter for iPhone,2020,04,30,21,4,17,False,True,The CCP’s Military-Civil Fusion strategy blurs...,SenTomCotton,NaN,NaN,NaN
1,1256037103173017603,Fri May 01 01:45:56 +0000 2020,Twitter for iPhone,2020,04,30,21,4,17,False,True,Tremendous work by @FreeBeacon \n\nWhile the r...,RichLowry,NaN,NaN,NaN
2,1256025982152200194,Fri May 01 01:01:44 +0000 2020,Twitter for iPhone,2020,04,30,21,4,17,False,True,Something seems rotten in Flynn's case — my co...,AndrewCMcCarthy,NaN,NaN,NaN
3,1256024436513411072,Fri May 01 00:55:36 +0000 2020,Twitter for iPhone,2020,04,30,20,4,17,False,True,.@BarackObama is Ultimately Responsible for @F...,BuckSexton,NaN,NaN,NaN
4,1256024228777857025,Fri May 01 00:54:46 +0000 2020,Twitter for iPhone,2020,04,30,20,4,17,False,False,NaN,NaN,“The only thing we got wrong is that it was ev...,156126.0,38037.0


In [54]:
save_data_csv(trump_df, 'trump_new.csv')